In [ ]:
import os
from env.audit import Audits

appsecret = os.getenv("admin_appsecret")
appid = os.getenv("admin_appid")
tenantid = os.getenv("admin_tenantid")

audit = Audits()
audit.set_ServicePrincipal(
    tenant_id=tenantid,
    client_id=appid,
    client_secret= appsecret
)


In [ ]:

# audit.set_ApplicationModules("Apps")
audit.set_ApplicationModules("Activity,Apps,Capacity,Catalog,Domains,FabricItems,Gateway,Graph,Refreshables,RefreshHistory,Roles,Tenant,Workspaces")
# audit.set_ApplicationModules("Activity,Apps")
# audit.set_ApplicationModules("Roles,Tenant,Workspaces")

# a list of the supported logging levels
import logging
Logging_levels = {
            "DEBUG": logging.DEBUG,
            "INFO": logging.INFO,
            "WARNING": logging.WARNING,
            "ERROR": logging.ERROR,
            "CRITICAL": logging.CRITICAL
}

audit.set_LakehouseName("FabricLake")
audit.set_PathInLakehouse("stage")
audit.set_WorkspaceName("FabricMonitor")
audit.set_logging_config("INFO", "brandon.log")

audit.set_Activity_cron("* * * * * 30")
audit.set_Catalog_cron("* * * * * 30")
audit.set_Capacity_cron("* * * * * 30")
audit.set_Apps_cron("* * * * * 30")
audit.set_Domains_cron("* * * * * 30") # Not yet implemented
audit.set_Graph_cron("* * * * * 30")
audit.set_Tenant_cron("* * * * * 30")
audit.set_RefreshHistory_cron("* * * * * 30")
audit.set_Refreshables_cron("* * * * * 30")
audit.set_Gateway_cron("* * * * * 30")

# print(audit.ServicePrincipal)
await audit.run()



In [ ]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from pyspark.sql.functions import lit
from pyspark import SparkConf


In [ ]:
sparkConf = SparkConf()
sparkConf.set("fs.azure.account.auth.type.onelake.dfs.core.windows.net", "OAuth")
sparkConf.set("fs.azure.account.oauth.provider.type.onelake.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
sparkConf.set("fs.azure.account.oauth2.client.id.onelake.dfs.core.windows.net", appid)
sparkConf.set("fs.azure.account.oauth2.client.secret.onelake.dfs.core.windows.net", appsecret)
sparkConf.set("fs.azure.account.oauth2.client.endpoint.onelake.dfs.core.windows.net", "https://login.microsoftonline.com/" + tenantid)
sparkConf.set("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.1")
spark = SparkSession.builder.config(conf=sparkConf).appName("SparkTest").getOrCreate()


schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", StringType(), True),
    StructField("timestamp", TimestampType(), True)
])

import datetime

now = datetime.datetime.now()

data = [("John", "30", now), ("Jane", "25", now)]
df = spark.createDataFrame(data, schema)
df = df.withColumn("source", lit("test"))

df.write.format("parquet").mode("append").save("abfss://onelake.dfs.core.windows.net/FabricLake.Lakehouse/Files/data.parquet")
# df.write.format("parquet").mode("append").save("FabricLake.Lakehouse/Files/data.parquet")
df.show()



In [ ]:
# df.write.format("delta").mode("append").saveAsTable("data")
df.write.format("parquet").save("data.parquet")

In [ ]:
# Instantiate a DataLakeServiceClient using a connection string
   from azure.storage.filedatalake import DataLakeServiceClient
   datalake_service_client = DataLakeServiceClient.from_connection_string(self.connection_string)

   # Instantiate a FileSystemClient
   file_system_client = datalake_service_client.get_file_system_client("mynewfilesystem")

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.storage.filedatalake import DataLakeServiceClient, FileSystemClient
from azure.identity import ClientSecretCredential
import datetime
import json

storage_account_name = "onelake"
workspace_name = "FabricMonitor"

credential = ClientSecretCredential(tenant_id=tenantid,
                            client_id=appid,
                            client_secret=appsecret)

file_system_client = FileSystemClient(
    account_url="https://onelake.dfs.fabric.microsoft.com",
    file_system_name=workspace_name,
    credential=credential)

dc = file_system_client.get_directory_client("FabricLake.Lakehouse/Files")
file_client = dc.get_file_client("data.parquet")
with open("./data.parquet", "rb") as data:
    file_client.upload_data(data, overwrite=True)

    


df.write.format("delta").mode("append").save("abfss://onelake.dfs.core.windows.net/FabricLake.Lakehouse/Files/data.parquet")
